In [47]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.autograd as autograd
import time
import os
import torchvision.utils as vutils

In [38]:
matplotlib_is_available = True
try:
  from matplotlib import pyplot as plt
except ImportError:
  print("Will skip plotting; matplotlib is not available.")
  matplotlib_is_available = False

In [39]:
# Data params
data_mean = 10
data_stddev = 5
use_gpu = torch.cuda.is_available()

In [40]:
# ### Uncomment only one of these to define what data is actually sent to the Discriminator
#(name, preprocess, d_input_func) = ("Raw data", lambda data: data, lambda x: x)
#(name, preprocess, d_input_func) = ("Data and variances", lambda data: decorate_with_diffs(data, 2.0), lambda x: x * 2)
#(name, preprocess, d_input_func) = ("Data and diffs", lambda data: decorate_with_diffs(data, 1.0), lambda x: x * 2)
(name, preprocess, d_input_func) = ("Only 4 moments", lambda data: get_moments(data), lambda x: 4)
print("Using data [%s]" % (name))

Using data [Only 4 moments]


In [41]:
def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))  # Gaussian

In [42]:
def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)  # Uniform-dist data into generator, _NOT_ Gaussian



In [43]:
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.sigmoid(self.map2(x))
        return self.map3(x)

In [44]:
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.elu(self.map2(x))
        # return F.sigmoid(self.map3(x))
        return self.map3(x)

In [45]:
def extract(v):
    return v.data.storage().tolist()


In [27]:
def stats(d):
    return [np.mean(d), np.std(d)]

In [28]:
def get_moments(d):
    # Return the first 4 moments of the data provided
    mean = torch.mean(d)
    diffs = d - mean
    var = torch.mean(torch.pow(diffs, 2.0))
    std = torch.pow(var, 0.5)
    zscores = diffs / std
    skews = torch.mean(torch.pow(zscores, 3.0))
    kurtoses = torch.mean(torch.pow(zscores, 4.0)) - 3.0  # excess kurtosis, should be 0 for Gaussian
    final = torch.cat((mean.reshape(1,), std.reshape(1,), skews.reshape(1,), kurtoses.reshape(1,)))
    return final


In [29]:
def decorate_with_diffs(data, exponent, remove_raw_data=False):

    mean = torch.mean(data.data, 1, keepdim=True)

    mean_broadcast = torch.mul(torch.ones(data.size()), mean.tolist()[0][0]).cuda()
    if use_gpu:
        mean_broadcast = mean_broadcast.cuda()
    diffs = torch.pow(data - Variable(mean_broadcast), exponent)
    if remove_raw_data:
        return torch.cat([diffs], 1)
    else:
        return torch.cat([data, diffs], 1)
    

In [30]:

# Model parameters

num_runs = 10

#job_name = "./saves/baseline/starter_gan_default"


g_input_size = 1     # Random noise dimension coming into generator, per output vector
g_hidden_size = 50   # Generator complexity
g_output_size = 1    # size of generated output vector
d_input_size = 100   # Minibatch size - cardinality of distributions
d_hidden_size = 50   # Discriminator complexity
d_output_size = 1    # Single dimension for 'real' vs. 'fake'
minibatch_size = d_input_size
d_learning_rate = 2e-4  # 2e-4
g_learning_rate = 2e-4
# optim_betas = (0.9, 0.999)
num_epochs = 30000
print_interval = 200
save_interval = 200
# d_steps = 1  # 'k' steps in the original GAN paper. Can put the discriminator on higher training freq than generator
d_steps = 5
g_steps = 1

dfe, dre, ge = 0, 0, 0
d_real_data, d_fake_data, g_fake_data = None, None, None

discriminator_activation_function = torch.sigmoid
generator_activation_function = torch.tanh



In [51]:
def plot_distribution(job_name, data_name, data):
    #print("Plotting the ", data_name, "...")
    #print(" Values: %s" % (str(gen_data)))
    fig = plt.hist(data, bins=50)
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.title('Histogram of ' + data_name + ' Distribution')
    plt.grid(True)
    #plt.savefig(job_name + data_name + '.png')
    plt.show()
    
   
    #plt.clf()

In [15]:
def train(job_name):
    #print(job_name)
    d_sampler = get_distribution_sampler(data_mean, data_stddev)
    gi_sampler = get_generator_input_sampler()
    G = Generator(input_size=g_input_size, hidden_size=g_hidden_size, output_size=g_output_size)
    D = Discriminator(input_size=d_input_func(d_input_size), hidden_size=d_hidden_size, output_size=d_output_size)
    # criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
    # d_optimizer = optim.Adam(D.parameters(), lr=d_learning_rate, betas=optim_betas)
    # g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=optim_betas)
    d_optimizer = optim.RMSprop(D.parameters(), lr=d_learning_rate)
    g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate)


    use_gpu = torch.cuda.is_available()
    #print("use_gpu: ", use_gpu)
    if use_gpu:
        G = G.cuda()
        D = D.cuda()
    start_time = time.time()
    if not os.path.exists(job_name + '_info_over_epoch.txt'):
        with open(job_name + '_info_over_epoch.txt', 'a'): pass
    open_file = open(job_name + '_info_over_epoch.txt', 'r+')
    open_file.seek(0)
    open_file.truncate()
    open_file.close()
    
    if not os.path.exists(job_name + '_results.txt'):
        with open(job_name + '_results.txt', 'a'): pass
    open_file = open(job_name + '_results.txt', 'r+')
    open_file.seek(0)
    open_file.truncate()
    open_file.close()
    
    avg_dre = 0
    avg_dfe = 0
    avg_ge = 0
    for epoch in range(num_epochs):
        for d_index in range(d_steps):
            # 1. Train D on real+fake
            D.zero_grad()
            #  1A: Train D on real
            d_real_data = Variable(d_sampler(d_input_size))
            if use_gpu:
                d_real_data = d_real_data.cuda()
            
            d_real_decision = D(preprocess(d_real_data))
            # d_real_error = criterion(d_real_decision, Variable(torch.ones(1)))  # ones = true
            d_real_error = -torch.mean(d_real_decision)
            d_real_error.backward() # compute/store gradients, but don't change params
            #  1B: Train D on fake
            d_gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            if use_gpu:
                d_gen_input = d_gen_input.cuda()
            d_fake_data = G(d_gen_input).detach()  # detach to avoid training G on these labels
            d_fake_decision = D(preprocess(d_fake_data.t()))
            # d_fake_error = criterion(d_fake_decision, Variable(torch.zeros(1)))  # zeros = fake
            d_fake_error = torch.mean(d_fake_decision)
            d_fake_error.backward()
            d_optimizer.step()     # Only optimizes D's parameters; changes based on stored gradients from backward()
            dre, dfe = d_real_decision.data[0], d_fake_decision.data[0]
            avg_dre += dre
            avg_dfe += dfe
            # Weight Clipping
            for p in D.parameters():
                p.data.clamp_(-0.01, 0.01)
        for g_index in range(g_steps):
            # 2. Train G on D's response (but DO NOT train D on these labels)
            G.zero_grad()
            gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            if use_gpu:
                gen_input = gen_input.cuda()
            g_fake_data = G(gen_input)
            dg_fake_decision = D(preprocess(g_fake_data.t()))
            # g_error = criterion(dg_fake_decision, Variable(torch.ones(1)))  # we want to fool, so pretend it's all genuine
            g_error = -torch.mean(dg_fake_decision)
            g_error.backward()
            g_optimizer.step()  # Only optimizes G's parameters
            ge = dg_fake_decision.data[0]
            avg_ge += ge
        
        if epoch % save_interval == 0:
            with open(job_name + '_info_over_epoch.txt', 'a') as f:
                f.write("{} {} {} {} {} {}\n".format(epoch, avg_dre/(d_steps*save_interval), avg_dfe/(d_steps*save_interval), avg_ge/(g_steps*save_interval), stats(extract(d_real_data)), stats(extract(d_fake_data))))
            avg_dre = 0
            avg_dfe = 0
            avg_ge = 0

    #Plot real and generated figures
    if matplotlib_is_available:
        plot_distribution(job_name, 'Real', d_real_data.cpu())
        plot_distribution(job_name, 'Generated', extract(g_fake_data))
        
    #Calculate and save KL_DIV
    input_data = g_fake_data.detach().cpu()
    target_data = d_real_data.view(100, 1).cpu()
    min_elem = abs(min(torch.min(input_data), torch.min(target_data))) + 1
    #print(min_elem)
    #min_tensor = Variable(torch.Tensor([min_elem])).expand(input_data.size())
    #print(min_tensor)
    #print(min_tensor.size())

    kl_loss = torch.nn.functional.kl_div(torch.log(input_data + min_elem), target_data + min_elem)
    gen_mean = extract(torch.mean(input_data, dim = 0))
    gen_std = extract(torch.std(input_data, dim = 0))
    real_mean = extract(torch.mean(target_data, dim = 0))
    real_std = extract(torch.std(target_data, dim = 0))
    
    
    with open(job_name + '_results.txt', 'a') as f:
                f.write("""Generated Mean: {} \nGenerated StdDev: {} \nReal Mean: {} \nReal StdDev: {}\nKL Loss: {} \n""".format(gen_mean, gen_std, real_mean, real_std, kl_loss))
                f.close()
        
    '''torch.save({
                'model_state_dict': G.state_dict(),
                'optimizer_state_dict': g_optimizer.state_dict()
                }, job_name + '_generator_model')'''
    
    print("Total run time: ", time.time() - start_time)
    
    return gen_mean[0], gen_std[0], real_mean[0], real_std[0], kl_loss
    
    
    
        
        

In [16]:
avg_gen_mean, avg_gen_std, avg_real_mean, avg_real_std, avg_kl_loss = 0, 0 ,0 ,0 ,0
path_name = "./saves/WL_tomaxent/WL_"
job_name = "/05_30_GAN_"
start_time = time.time()
for i in range(num_runs):
    new_path = path_name + str(i)
    os.mkdir(new_path)
    curr_job = new_path+job_name
    gen_mean, gen_std, real_mean, real_std, kl_loss = train(curr_job)
    print(f'--------- Run #{i} --------------')
    print(f"Generated Mean: {gen_mean}\nGenerated StdDev: {gen_std}\nReal_Mean: {real_mean}\nReal StdDev: {real_std}\nKL_Loss: {kl_loss}")
    
    avg_gen_mean += gen_mean
    avg_gen_std += gen_std
    avg_real_mean += real_mean
    avg_real_std += real_std
    avg_kl_loss += kl_loss
avg_gen_mean /= num_runs
avg_gen_std /= num_runs
avg_real_mean /= num_runs
avg_real_std /= num_runs
avg_kl_loss /= num_runs

final_data = "./saves/WL_final_results.txt"
'''if not os.path.exists(final_data):
    with open(final_data, 'a'): pass
open_file = open(final_data, 'r+')
open_file.seek(0)
open_file.truncate()
open_file.close()
'''
with open(final_data, 'a') as f:
    f.write("""Generated Mean: {} \nGenerated StdDev: {} \nReal Mean: {} \nReal StdDev: {}\nKL Loss: {} \n""".format(avg_gen_mean, avg_gen_std, avg_real_mean, avg_real_std, avg_kl_loss))
    f.close()
print("Total run time: ", time.time() - start_time)


FileExistsError: [Errno 17] File exists: './saves/WL_tomaxent/WL_0'

In [32]:
'''checkpoint=None
if os.path.isfile(job_name + '_generator_model'):
    print("fetching model")
    checkpoint = torch.load(job_name + '_generator_model')
    G.load_state_dict(checkpoint['model_state_dict'])
    g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=(beta1, 0.999))
    g_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])'''

'''G.eval().cpu()
with torch.no_grad():
    gen_input = Variable(gi_sampler(minibatch_size, g_input_size))

    g_fake_data = G(gen_input)'''


#plot_distribution('Generated', extract(g_fake_data))

d_sampler = get_distribution_sampler(data_mean, data_stddev)

d_real_data = Variable(d_sampler(d_input_size))
x = torch.min(d_real_data)
print(x)



tensor(-2.8913)


In [ ]:
test_sampler = get_distribution_sampler(100, 1)
input_1 = Variable(test_sampler(d_input_size))
input_2 = Variable(test_sampler(d_input_size))

kl_loss = torch.nn.functional.kl_div(torch.log(input_1), input_2)

print(kl_loss)


In [ ]:
data_mean = 10
data_stddev = 5
# Model params
g_input_size = 1     # Random noise dimension coming into generator, per output vector
g_hidden_size = 50   # Generator complexity
g_output_size = 1    # size of generated output vector
d_input_size = 100   # Minibatch size - cardinality of distributions
d_hidden_size = 50   # Discriminator complexity
d_output_size = 1    # Single dimension for 'real' vs. 'fake'
minibatch_size = d_input_size
d_learning_rate = 2e-4  # 2e-4
g_learning_rate = 2e-4
# optim_betas = (0.9, 0.999)
num_epochs = 30000
print_interval = 200
# d_steps = 1  # 'k' steps in the original GAN paper. Can put the discriminator on higher training freq than generator
d_steps = 5
g_steps = 1
# ### Uncomment only one of these
#(name, preprocess, d_input_func) = ("Raw data", lambda data: data, lambda x: x)
#(name, preprocess, d_input_func) = ("Data and variances", lambda data: decorate_with_diffs(data, 2.0), lambda x: x * 2)
print("Using data [%s]" % (name))
# ##### DATA: Target data and generator input data
def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))  # Gaussian
def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)  # Uniform-dist data into generator, _NOT_ Gaussian
# ##### MODELS: Generator model and discriminator model
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.sigmoid(self.map2(x))
        return self.map3(x)
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.elu(self.map2(x))
        # return F.sigmoid(self.map3(x))
        return self.map3(x)
def extract(v):
    return v.data.storage().tolist()
def stats(d):
    return [np.mean(d), np.std(d)]
def decorate_with_diffs(data, exponent):
    mean = torch.mean(data.data, 1)
    mean_broadcast = torch.mul(torch.ones(data.size()), mean.tolist()[0][0])
    diffs = torch.pow(data - Variable(mean_broadcast), exponent)
    return torch.cat([data, diffs], 1)
d_sampler = get_distribution_sampler(data_mean, data_stddev)
gi_sampler = get_generator_input_sampler()
G = Generator(input_size=g_input_size, hidden_size=g_hidden_size, output_size=g_output_size)
D = Discriminator(input_size=d_input_func(d_input_size), hidden_size=d_hidden_size, output_size=d_output_size)
# criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
# d_optimizer = optim.Adam(D.parameters(), lr=d_learning_rate, betas=optim_betas)
# g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=optim_betas)
#d_optimizer = optim.RMSprop(D.parameters(), lr=d_learning_rate)
#g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate)
lr = 5e-5
betas = (.9, .99)
G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=betas)
D_optimizer = optim.Adam(D.parameters(), lr=lr, betas=betas)

one = torch.FloatTensor([1])
mone = one * -1
#if use_cuda:
    #one = one.cuda()
    #mone = mone.cuda()
    
LAMBDA = .1  # Smaller lambda seems to help for toy tasks specifically
    
def calc_gradient_penalty(netD, real_data, fake_data):
    alpha = torch.rand(1, minibatch_size)
    fake_data = fake_data.view(1, 100)
    #print(alpha.size())
    #print(real_data.size())
    #alpha = alpha.expand(real_data.size())
    #print((alpha*real_data).size())
    #print(((1 - alpha) * fake_data).size())

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)
    #print(interpolates.size())


    interpolates = Variable(interpolates, requires_grad=True)
    #print("Interpolates size: ", interpolates.size())

    disc_interpolates = netD(preprocess(interpolates))

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty



for epoch in range(num_epochs):
    for d_index in range(d_steps):
        
        #  1A: Train D on real
        d_real_data = Variable(d_sampler(d_input_size))

        
        # 1. Train D on real+fake
        D.zero_grad()
        
        d_real_decision = D(preprocess(d_real_data))
        # d_real_error = criterion(d_real_decision, Variable(torch.ones(1)))  # ones = true
        d_real_error = -torch.mean(d_real_decision)
        d_real_error.backward() # compute/store gradients, but don't change params
        
        
        #  1B: Train D on fake
        d_gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
        noisev = Variable(d_gen_input, volatile=True)
        
        d_fake_data = G(d_gen_input).detach()  # detach to avoid training G on these labels
        d_fake_decision = D(preprocess(d_fake_data.t()))
        # d_fake_error = criterion(d_fake_decision, Variable(torch.zeros(1)))  # zeros = fake
        d_fake_error = torch.mean(d_fake_decision)
        d_fake_error.backward()
        
        
        gradient_penalty = calc_gradient_penalty(D, d_real_data.data, d_fake_data.data)
        gradient_penalty.backward()
        D_optimizer.step()     # Only optimizes D's parameters; changes based on stored gradients from backward()
       
    for g_index in range(g_steps):
        # 2. Train G on D's response (but DO NOT train D on these labels)
        G.zero_grad()
        gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
        g_fake_data = G(gen_input)
        dg_fake_decision = D(preprocess(g_fake_data.t()))
        # g_error = criterion(dg_fake_decision, Variable(torch.ones(1)))  # we want to fool, so pretend it's all genuine
        g_error = -torch.mean(dg_fake_decision)
        g_error.backward()
        G_optimizer.step()  # Only optimizes G's parameters
    if epoch % print_interval == 0:
        print("%s: D: %s/%s G: %s (Real: %s, Fake: %s) " % (epoch,
                                                            extract(d_real_error)[0],
                                                            extract(d_fake_error)[0],
                                                            extract(g_error)[0],
                                                            stats(extract(d_real_data)),
                                                            stats(extract(d_fake_data))))
        
        
plot_distribution("pls", 'improved_wgan', extract(g_fake_data))


Using data [Only 4 moments]
0: D: -0.6192820072174072/0.4124716520309448 G: -0.4157402515411377 (Real: [9.662786051034928, 5.083345870989862], Fake: [0.416041356921196, 0.0002415919145685649]) 


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:126: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


200: D: -17.70221710205078/0.32307878136634827 G: -0.3288811445236206 (Real: [9.790859960913657, 4.714276550878154], Fake: [0.5091559797525406, 0.0023915768024622024]) 
400: D: -55.18533706665039/-4.113702297210693 G: 3.8627030849456787 (Real: [9.753457130491734, 5.1475163237831305], Fake: [0.6164691197872162, 0.003918830354412939]) 
600: D: -122.57506561279297/-11.879233360290527 G: 12.546199798583984 (Real: [9.479424178898334, 5.612711930947127], Fake: [0.8557269984483719, 0.005899816017178743]) 
800: D: -218.66624450683594/-18.52759552001953 G: 20.546100616455078 (Real: [9.86254470050335, 4.637921983996538], Fake: [1.2335774099826813, 0.005960086944045227]) 
1000: D: -384.3892517089844/-25.36939239501953 G: 17.203201293945312 (Real: [10.520494924783707, 5.724990121503178], Fake: [1.325110809803009, 0.0008902662594323366]) 
1200: D: -542.8588256835938/-32.019866943359375 G: 33.53192138671875 (Real: [9.718663757145405, 4.8713512639980046], Fake: [1.3403606808185577, 0.0015697920421282

10600: D: -1683.976318359375/1895.9185791015625 G: -1880.445556640625 (Real: [10.214170987606048, 5.283341349941495], Fake: [13.471195974349975, 6.618534263920208]) 
10800: D: -1082.7882080078125/1314.702392578125 G: -1286.2401123046875 (Real: [9.346726776510478, 5.132981975460081], Fake: [14.671590852737427, 6.910357497027585]) 
11000: D: -699.8746337890625/784.26953125 G: -767.029296875 (Real: [9.125654568076135, 4.829569225560545], Fake: [14.29046569943428, 7.341357480183599]) 
11200: D: -342.343017578125/397.8214111328125 G: -438.2140808105469 (Real: [10.169466665014625, 4.920132938180462], Fake: [12.937006783485412, 8.153200779891277]) 
11400: D: -138.67245483398438/96.8550796508789 G: -150.0624542236328 (Real: [10.368147209882736, 4.896909083438871], Fake: [12.129037162959575, 8.349220649160927]) 
11600: D: -167.85040283203125/33.21993637084961 G: -25.018959045410156 (Real: [9.672796524167062, 4.514977946669096], Fake: [11.772662533819675, 7.9804687000529135]) 
11800: D: -43.7341